<a href="https://colab.research.google.com/github/sadrireza/Neural-Networks/blob/main/Callbacks_%26_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#having callbacks is like to have drones instead of paperplanes.
#you have control over drones but no control over paperplanes after their launching
#callbacks help us transform model.fit() fro paper airplane into autonomous drone
#keras.callbacks.ModelCheckpoint
#keras.callbacks.EarlyStoping
#keras.callbacks.LearningRateScheduler
#keras.callbacks.ReduceLRonPlateau
#keras.callbacks.CSVLogger

In [26]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import regularizers

In [17]:
%%capture
(images, labels), (test_images, test_labels) = mnist.load_data()

In [18]:
#Reshaping (Vectorization), Normalization, Producing validation data and labels
images = images.reshape((60000,28*28)).astype('float32') / 255
train_images, val_images = images [10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [20]:
def get_mnist_model():
  inputs = layers.Input(shape=(28*28,))
  x = layers.Dense(256, activation = 'relu')(inputs)
  outputs = layers.Dense(10, activation = 'softmax')(x)

  model = keras.Model(inputs,outputs)
  return model

In [23]:
def get_mnist_model_regularized():
  inputs = layers.Input(shape=(28*28,))
  x = layers.Dense(256, activation = 'relu',
                   kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(inputs)
  #Regularization: 20 percent of neurons will be dropped out
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(10, activation = 'relu',
                   kernel_regularizer=regularizers.L2(1e-4))(x)
  #coming from weight regularizers part of the keras website, we bring l1/l2 regularization code
  outputs = layers.Dense(10, activation = 'softmax')(x)

  model = keras.Model(inputs,outputs)
  return model

In [27]:
#model = get_mnist_model()
model = get_mnist_model_regularized()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │         200,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           2,570 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 10)                  │             110 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 203,640 (795.47 KB)

 Trainable params: 203,640 (795.47 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [29]:
#Using Callbacks
#each callback can be made as a variable

# an address is taken and after each epoch if conditions are met, model will be saved
#save_best_only: when you want to save only the best version (if you put false, all versions will be saved)
model_checkpoint = keras.callbacks.ModelCheckpoint('checkpoint.keras', monitor='accuracy', save_best_only=True)

#earlystopping: whenever model begins to overfit, training willbe stopped
#patience=2 if for 2 epochs, monitor is reported wrong, stop training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)

#a log file is made and all loss and metrics values are saved line by line inside it
logger = keras.callbacks.CSVLogger('logg.csv')

callback_list = [model_checkpoint, early_stopping, logger]

In [30]:
model.fit(train_images, train_labels, epochs=10, validation_data = (val_images, val_labels), callbacks = callback_list)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.8150 - loss: 0.7041 - val_accuracy: 0.9579 - val_loss: 0.2326
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9541 - loss: 0.2339 - val_accuracy: 0.9610 - val_loss: 0.2127
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9651 - loss: 0.2010 - val_accuracy: 0.9626 - val_loss: 0.2090
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9695 - loss: 0.1837 - val_accuracy: 0.9742 - val_loss: 0.1725
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9727 - loss: 0.1729 - val_accuracy: 0.9720 - val_loss: 0.1787
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9736 - loss: 0.1664 - val_accuracy: 0.9717 - val_loss: 0.1809
